In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [136]:
df = pd.read_csv('./dataset/delivery_raw.csv',sep='\t')
df.head()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift,total_busy,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration
0,1.0,2015-02-06 22:24:17,2015-02-06 23:27:16,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0
1,2.0,2015-02-10 21:49:25,2015-02-10 22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0
2,3.0,2015-01-22 20:39:28,2015-01-22 21:09:09,5477,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0
3,3.0,2015-02-03 21:21:45,2015-02-03 22:13:00,5477,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0
4,3.0,2015-02-15 02:40:36,2015-02-15 03:20:26,5477,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0


In [106]:
df['order_protocol'].unique()

array([ 1.,  2.,  3.,  4., nan,  5.,  6.,  7.])

In [44]:
df.shape

(197428, 16)

In [45]:
df.isnull().sum()

market_id                                         987
created_at                                          0
actual_delivery_time                                7
store_id                                            0
store_primary_category                           4760
order_protocol                                    995
total_items                                         0
subtotal                                            0
num_distinct_items                                  0
min_item_price                                      0
max_item_price                                      0
total_onshift                                   16262
total_busy                                      16262
total_outstanding_orders                        16262
estimated_order_place_duration                      0
estimated_store_to_consumer_driving_duration      526
dtype: int64

시간 속성
- market_id: 지역(배달이 이루어지는 도시) 아이디
- created_at: 주문이 생성된 시간의 Timestamp(UTC)
- actual_delivery_time: 주문자가 배달을 받은 시간의 Timestamp(UTC)  
  
식당 속성
- store_id: 식당 아이디
- store_primary_category: 식당의 카테고리(italian, asian 등)
- order_protocol: 주문을 받을 수 있는 방식을 나타내는 아이디  

주문 속성
- total_items: 주문에 포함된 아이템(음식) 개수
- subtotal: 가격(센트 단위)
- num_distinct_items: 주문에 포함된 비중복 아이템 개수
- min_item_price: 주문에 포함된 아이템 중 가장 싼 아이템의 가격
- max_item_price: 주문에 포함된 아이템 중 가장 비싼 아이템의 가격  

지역 상황 속성
- total_onshift: 주문이 생성되었을 때 가게로부터 10마일 이내에 있는 배달원들의 수
- total_busy: 위 배달원들 중 주문에 관여하고 있는 사람들의 수
- total_outstanding_orders: 주문한 가게로부터 10마일 이내에 있는 다른 주문들의 수  

다른 모델들의 예측값
- estimated_order_place_duration: 식당이 주문을 받을 때까지 걸릴 것으로 예상되는 시간(초단위)
- estimated_store_to_consumer_driving_duration: 식당에서 출발해 주문자에 도착할 때까지 걸릴 것으로 예측되는 시간(초단위)

In [74]:
df[df['store_primary_category'].isnull()]['store_id'].value_counts()

791     180
257     123
3458    122
4937    121
4717    109
       ... 
2608      1
4924      1
2924      1
4779      1
2956      1
Name: store_id, Length: 1111, dtype: int64

In [70]:
storeid = df[df['store_primary_category'].isnull()]['store_id'].unique()
storeid

array([5477, 4149, 3879, ..., 2177, 1340, 2956], dtype=int64)

In [105]:
df[df['store_id'] == 3458]['store_primary_category'].unique()

array([nan, 'american', 'pizza', 'other'], dtype=object)

하나의 식당에서도 여러 종류의 음식을 파는 것을 알 수 있다.  
store_id로 store_primary_category를 채울 수는 없을 것 같다.

In [137]:
df['created_at'] = pd.to_datetime(df['created_at'])
df['actual_delivery_time'] = pd.to_datetime(df['actual_delivery_time'])

In [18]:
df.head()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift,total_busy,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration
0,1.0,2015-02-06 22:24:17,2015-02-06 23:27:16,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0
1,2.0,2015-02-10 21:49:25,2015-02-10 22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0
2,3.0,2015-01-22 20:39:28,2015-01-22 21:09:09,5477,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0
3,3.0,2015-02-03 21:21:45,2015-02-03 22:13:00,5477,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0
4,3.0,2015-02-15 02:40:36,2015-02-15 03:20:26,5477,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0


In [108]:
df.corr()

,market_id,store_id,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift,total_busy,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,delivery_time
market_id,1.000000,0.017303,-0.021898,-0.006858,-0.016230,0.002553,-0.000145,-0.004911,0.069769,0.060505,0.063767,-0.053240,0.006610,-0.005784
store_id,0.017303,1.000000,0.019553,-0.008783,-0.013362,-0.012718,-0.009140,-0.018660,-0.023106,-0.023215,-0.020307,0.017472,0.001964,-0.000394
order_protocol,-0.021898,0.019553,1.000000,0.008801,-0.053610,-0.023329,-0.045415,-0.090955,0.145055,0.149608,0.134938,-0.678219,-0.008305,-0.006642
total_items,-0.006858,-0.008783,0.008801,1.000000,0.558061,0.763906,-0.393146,-0.058235,0.031705,0.028578,0.034348,-0.023661,0.004678,0.004926
subtotal,-0.016230,-0.013362,-0.053610,0.558061,1.000000,0.681099,0.037041,0.505545,0.130810,0.125547,0.130080,0.039023,0.034646,0.011213
num_distinct_items,0.002553,-0.012718,-0.023329,0.763906,0.681099,1.000000,-0.448738,0.041871,0.065920,0.060618,0.067771,0.003487,0.022680,0.006764
min_item_price,-0.000145,-0.009140,-0.045415,-0.393146,0.037041,-0.448738,1.000000,0.545485,0.042387,0.043900,0.041021,0.053070,0.003372,0.004728
max_item_price,-0.004911,-0.018660,-0.090955,-0.058235,0.505545,0.041871,0.545485,1.000000,0.133112,0.131006,0.130675,0.085275,0.026581,0.009389
total_onshift,0.069769,-0.023106,0.145055,0.031705,0.130810,0.065920,0.042387,0.133112,1.000000,0.943787,0.936119,-0.185682,0.045889,0.046417
total_busy,0.060505,-0.023215,0.149608,0.028578,0.125547,0.060618,0.043900,0.131006,0.943787,1.000000,0.932913,-0.191833,0.044474,0.059995


In [114]:
df[['delivery_time','store_primary_category']].groupby('store_primary_category').mean()

,delivery_time
store_primary_category,
afghan,57.058824
african,55.000000
alcohol,49.897297
alcohol-plus-food,60.000000
american,52.790866
...,...
thai,53.593080
turkish,49.493671
vegan,52.867384


In [117]:
df[df['delivery_time'] == 141950]

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift,total_busy,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,delivery_time
2690,1.0,2014-10-19 05:24:15,2015-01-25 19:11:54,3560,italian,1.0,1,1695,1,1595,1595,NaN,NaN,NaN,446,412.0,141950


In [116]:
df.describe()

,market_id,store_id,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift,total_busy,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,delivery_time
count,196434.000000,197421.000000,196426.000000,197421.000000,197421.000000,197421.000000,197421.000000,197421.000000,181159.000000,181159.000000,181159.000000,197421.000000,196895.000000,197421.000000
mean,2.978700,3530.581397,2.882358,3.196367,2682.326379,2.670780,686.224596,1159.590444,44.806866,41.738787,58.047969,308.560244,545.357089,53.474250
std,1.524879,2053.493429,1.503775,2.666552,1823.106256,1.630261,522.044061,558.416236,34.525913,32.145163,52.660056,90.139725,219.353976,320.501579
min,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,-86.000000,0.000000,-4.000000,-5.000000,-6.000000,0.000000,0.000000,10.000000
25%,2.000000,1686.000000,1.000000,2.000000,1400.000000,1.000000,299.000000,800.000000,17.000000,15.000000,17.000000,251.000000,382.000000,40.000000
50%,3.000000,3592.000000,3.000000,3.000000,2200.000000,2.000000,595.000000,1095.000000,37.000000,34.000000,41.000000,251.000000,544.000000,50.000000
75%,4.000000,5299.000000,4.000000,4.000000,3395.000000,3.000000,949.000000,1395.000000,65.000000,62.000000,85.000000,446.000000,702.000000,60.000000
max,6.000000,6987.000000,7.000000,411.000000,27100.000000,20.000000,14700.000000,14700.000000,171.000000,154.000000,285.000000,2715.000000,2088.000000,141950.000000


## 데이터 정제

In [134]:
# eliminate 
d_idx = df[df['actual_delivery_time'].isnull()].index
df = df.drop(d_idx)

In [131]:
import math
def categorize_delivery_time(df):
    '''
    categorize_delivery_time by 10 minutes
    13 -> 20
    45 -> 50
    '''
    df['delivery_time'] = df['actual_delivery_time'] - df['created_at']
    df['delivery_time'] = df['delivery_time'].dt.total_seconds().div(60).astype(float)
    df['delivery_time'] = df['delivery_time'].apply(lambda x: math.ceil(x*0.1)*10)
    return df

In [132]:
from sklearn.impute import SimpleImputer
def impute_total_value(df):
    '''
    impute kinds of total_ values with median
    use sklearn.imputer
    '''
    imp = SimpleImputer(strategy='median')
    df['total_onshift'] = imp.fit_transform(df[['total_onshift']])
    df['total_busy'] = imp.fit_transform(df[['total_busy']])
    df['total_outstanding_orders'] = imp.fit_transform(df[['total_outstanding_orders']])
    return df

In [138]:
df = impute_total_value(df)
df = df.dropna()
df = categorize_delivery_time(df)
df.isnull().sum()

,market_id,store_id,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift,total_busy,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,delivery_time
market_id,1.000000,0.020166,-0.020602,-0.004932,-0.015630,0.004832,-0.003561,-0.007894,0.029245,0.019603,0.010191,-0.057419,0.008067,-0.005870
store_id,0.020166,1.000000,0.015154,-0.008538,-0.012474,-0.012276,-0.009432,-0.018110,-0.020848,-0.021297,-0.017698,0.023921,0.001042,-0.000367
order_protocol,-0.020602,0.015154,1.000000,0.009974,-0.052672,-0.021552,-0.046186,-0.091392,0.144036,0.148551,0.134452,-0.675909,-0.008811,-0.006658
total_items,-0.004932,-0.008538,0.009974,1.000000,0.555445,0.762339,-0.393201,-0.057767,0.031652,0.028826,0.034579,-0.024936,0.003692,0.004738
subtotal,-0.015630,-0.012474,-0.052672,0.555445,1.000000,0.680701,0.036823,0.506981,0.127933,0.123148,0.127786,0.037761,0.034608,0.011017
num_distinct_items,0.004832,-0.012276,-0.021552,0.762339,0.680701,1.000000,-0.449288,0.042865,0.063661,0.058658,0.065735,0.001875,0.021942,0.006591
min_item_price,-0.003561,-0.009432,-0.046186,-0.393201,0.036823,-0.449288,1.000000,0.543794,0.040427,0.041975,0.039079,0.053424,0.004173,0.004812
max_item_price,-0.007894,-0.018110,-0.091392,-0.057767,0.506981,0.042865,0.543794,1.000000,0.129398,0.127525,0.127061,0.084597,0.027782,0.009372
total_onshift,0.029245,-0.020848,0.144036,0.031652,0.127933,0.063661,0.040427,0.129398,1.000000,0.943972,0.936151,-0.179704,0.043721,0.003356
total_busy,0.019603,-0.021297,0.148551,0.028826,0.123148,0.058658,0.041975,0.127525,0.943972,1.000000,0.933006,-0.185422,0.042487,0.004422


In [139]:
df.head()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift,total_busy,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,delivery_time
0,1.0,2015-02-06 22:24:17,2015-02-06 23:27:16,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0,70
1,2.0,2015-02-10 21:49:25,2015-02-10 22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0,70
8,2.0,2015-02-16 00:11:35,2015-02-16 00:38:01,5477,indian,3.0,4,4771,3,820,1604,8.0,6.0,18.0,446,289.0,30
14,1.0,2015-02-12 03:36:46,2015-02-12 04:14:39,2841,italian,1.0,1,1525,1,1525,1525,5.0,6.0,8.0,446,795.0,40
15,1.0,2015-01-27 02:12:36,2015-01-27 03:02:24,2841,italian,1.0,2,3620,2,1425,2195,5.0,5.0,7.0,446,205.0,50


In [ ]:
import sklearn as skn
from sklearn.model_selection import train_test_split


In [ ]:
# 학습에 사용할 컬럼선택
df_train = df[['','','']]
y = df['delivery_time']

# train, test split
X_train, y_train, X_test, y_test = train_test_split(df_train, y, test_size=0.1, random_state=42)


## 학습